# Neural Network Architectures

In [5]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import tensorflow as tf 

from tensorflow.keras.applications import resnet50
# from tensorflow.keras.applications.densenet import preprocess_input

In [8]:
resnet = resnet50.ResNet50()

102967424/102967424 [==============================] - 86s 1us/step


In [10]:
resnet.summary()

Model: "resnet50"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 224, 224, 3  0           []                               
                                )]                                                                
                                                                                                  
 conv1_pad (ZeroPadding2D)      (None, 230, 230, 3)  0           ['input_1[0][0]']                
                                                                                                  
 conv1_conv (Conv2D)            (None, 112, 112, 64  9472        ['conv1_pad[0][0]']              
                                )                                                                 
                                                                                           

# Preprocessing steps

In [27]:
jpeg = tf.io.read_file("C:/Users/as/Desktop/Deep Learning/Deep-Learning/data/natural_images/cat/cat_0000.jpg")
cat_image = tf.image.decode_jpeg(jpeg)
cat_image_resized = tf.cast(tf.image.resize(cat_image,(224,224)), tf.int64)

In [28]:
cat_image_preprocessed = resnet50.preprocess_input(cat_image_resized)

In [29]:
cat_image_batch = tf.expand_dims(cat_image_preprocessed,0)

In [31]:
resnet50.decode_predictions(resnet.predict(cat_image_batch),top=100)

1/1 [==============================] - 0s 131ms/step


[[('n02123045', 'tabby', 0.84536225),
  ('n02123159', 'tiger_cat', 0.056560062),
  ('n02124075', 'Egyptian_cat', 0.035799548),
  ('n02123394', 'Persian_cat', 0.02686179),
  ('n03223299', 'doormat', 0.008021036),
  ('n02127052', 'lynx', 0.0028128908),
  ('n06785654', 'crossword_puzzle', 0.0017352969),
  ('n02870880', 'bookcase', 0.0015343983),
  ('n03958227', 'plastic_bag', 0.0010669724),
  ('n04589890', 'window_screen', 0.0010359293),
  ('n04201297', 'shoji', 0.00088329014),
  ('n04040759', 'radiator', 0.0008524199),
  ('n02971356', 'carton', 0.0008394766),
  ('n03131574', 'crib', 0.0007623523),
  ('n03085013', 'computer_keyboard', 0.0007481795),
  ('n04265275', 'space_heater', 0.0007406857),
  ('n03127925', 'crate', 0.00058532105),
  ('n03793489', 'mouse', 0.0005615425),
  ('n04004767', 'printer', 0.0005347966),
  ('n04033995', 'quilt', 0.00045369618),
  ('n03598930', 'jigsaw_puzzle', 0.00044814972),
  ('n04204238', 'shopping_basket', 0.00039523377),
  ('n04152593', 'screen', 0.000363